### <p style="font-family:Garamond,serif">Following the steps in NLP course notebook week 2.2, I first conduct LDA to the tweets I scrapped from #AstraZeneca and #Pfizer, then continue on generation visuialtion with pyLDAvis.</p>

In [3]:
!pip install pyldavis
import matplotlib.pyplot as plt
from wordcloud import WordCloud as wc
import pandas as pd 
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction import _stop_words
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize.casual import casual_tokenize
import pandas as pd
from nlpia.data.loaders import get_data
from sklearn.decomposition import TruncatedSVD

In [4]:
import re 
import nltk 
import numpy as np

In [5]:
from __future__ import print_function

In [6]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [9]:
import pyLDAvis

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
file_path = '/Users/chenxifang/Documents/GitHub/helenchennlp/data/en_hashtag_pfizer 2.csv'
data_from_csv = pd.read_csv(file_path, encoding='utf-8') #Reads a CSV file into a pandas dataframe
data_from_csv.head(20) #show us the first 3 rows

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


text
0     I got my #Pfizer booster shot today and the ...
1     If the U.S. would release the vaccine, deman...
2   Happy to get my #Pfizer booster on Saturday af...
3     Is the U.S. Treasury heavy in #Pfizer which ...
4     They can always reverse it at a later date. ...
5     ‘Brook Jackson, a regional director who was ...
6     Once you give a company with tentacles in ev...
7     Relax, that’s a donut not a booster shot. Yo...
8     Managed a successful #covidvaccine pop up to...
9     Which vaccine is the best booster for #astra...
10    #auspol … and #China #War will take the focu...
11    Hey at @ScottMorrisonMP we have a bucket loa...
12    If #omicron is so bad…why isn’t Biden fast t...
13                    Why is #Pfizer stock dropping?.
14    REMINDER:\n\nALL of the injections available...
15    The virus at the moment is millions who can’...
16  2 hours later and I'm still waiting for the th...
17  Omicron Variant B.1.1.529 has spike proteins -...
18  Shot #2 coming up for the 5-11 year-old in our...
19    Pfizer CEO Bourla: If necessary, the supply ...

In [11]:
np.shape(data_from_csv) 

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(80104, 1)

In [12]:
df = data_from_csv.rename(columns={'text': 'text'})
df

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


text
0        I got my #Pfizer booster shot today and the ...
1        If the U.S. would release the vaccine, deman...
2      Happy to get my #Pfizer booster on Saturday af...
3        Is the U.S. Treasury heavy in #Pfizer which ...
4        They can always reverse it at a later date. ...
...                                                  ...
80099    The two #Pfizer scientists responsible for t...
80100    Stock markets: the big fat #Coronarotation\n...
80101    As I have long said, #Pfizer and the others ...
80102    Sure is interesting timing that the announce...
80103  #Pfizer waited for a few days after the electi...

[80104 rows x 1 columns]

In [13]:
#Louis' tokenizer 
lem = WordNetLemmatizer()

def my_tokeniser(doc):
    #Split on spaces
    tokens = re.split(r'[-\s.,;!?]+', doc)
    processed = []
    for t in tokens:
        #Lemmatise and make lowercase
        #Can try changing this line to see how it impacts results (e.g., remove lemmatising, keep lowercase)
        t = lem.lemmatize(t.lower()) 
        #Remove stop words
        if not t in _stop_words.ENGLISH_STOP_WORDS:
            processed = processed + [t]
    #Return an array of tokens for that document
    return processed

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
#Create count vectors
cv = CountVectorizer(min_df=5, tokenizer=my_tokeniser,)

count_array = cv.fit_transform(df['text'].values.astype('U'))
count_array

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<80104x14966 sparse matrix of type '<class 'numpy.int64'>'
	with 1191910 stored elements in Compressed Sparse Row format>

In [15]:
#create tfidf vectors from count vectors
transformer = TfidfTransformer()
transformer.fit(count_array)
tfidf = transformer.transform(count_array)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
tfidf_vectorizer = TfidfVectorizer(**cv.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(df['text'].values.astype('U'))
print(dtm_tfidf.shape)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:1805: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  warnings.warn("Only {} 'dtype' should be used. {} 'dtype' will "


(80104, 14966)


In [17]:
tfidf_docs = tfidf.toarray()
tfidf_docs

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[0.11038607, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04361426, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.09560469, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04390761, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [18]:
len(cv.vocabulary_)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


14966

In [19]:
cv.vocabulary_

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'': 0,
 'got': 8603,
 '#pfizer': 1810,
 'booster': 6062,
 'shot': 12536,
 'today': 13449,
 'setup': 12443,
 'place': 11213,
 'thinking': 13354,
 'watch': 14130,
 'episode': 7866,
 'tonight': 13475,
 'u': 13664,
 's': 12208,
 'release': 11911,
 'vaccine': 13878,
 'demand': 7253,
 'pharmaceutical': 11162,
 'supply': 13080,
 'country': 6962,
 'afford': 5293,
 'cut': 7093,
 'variant': 13934,
 'happening': 8765,
 'world': 14331,
 'greed': 8667,
 'kill': 9682,
 '@potus': 4119,
 '#covid19': 584,
 '#moderna': 1529,
 'happy': 8770,
 'saturday': 12269,
 'afternoon': 5306,
 "it's": 9502,
 'monday': 10381,
 'arm': 5578,
 'soreness': 12741,
 'ha': 8720,
 'subsided': 13015,
 'wa': 14065,
 'tired': 13431,
 'joint': 9601,
 'achy': 5195,
 '2': 3258,
 'day': 7151,
 'weird': 14187,
 'rash': 11740,
 'armpit': 5587,
 'jab': 9528,
 'fading': 8091,
 'temporary': 13262,
 'discomfort': 7426,
 'greater': 8662,
 'good': 8587,
 'worth': 14345,
 '\\n#vaxxed': 4406,
 'treasury': 13564,
 'heavy': 8850,
 'they’re': 

In [21]:
# Get the column_numbers for each term in our vocabulary
column_nums, terms = zip(*sorted(zip(cv.vocabulary_.values(), cv.vocabulary_.keys())))
tfidf_docs_df = pd.DataFrame(tfidf_docs, columns=terms, index=df.index)
tfidf_docs_df #show it

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"  "#pfizer  "#pfizer"  "\n  "\n#pfizer  "\n\n  \
0      0.110386  0.0       0.0        0.0  0.0         0.0    0.0   
1      0.043614  0.0       0.0        0.0  0.0         0.0    0.0   
2      0.000000  0.0       0.0        0.0  0.0         0.0    0.0   
3      0.106912  0.0       0.0        0.0  0.0         0.0    0.0   
4      0.108103  0.0       0.0        0.0  0.0         0.0    0.0   
...         ...  ...       ...        ...  ...         ...    ...   
80099  0.053307  0.0       0.0        0.0  0.0         0.0    0.0   
80100  0.118041  0.0       0.0        0.0  0.0         0.0    0.0   
80101  0.095605  0.0       0.0        0.0  0.0         0.0    0.0   
80102  0.043908  0.0       0.0        0.0  0.0         0.0    0.0   
80103  0.000000  0.0       0.0        0.0  0.0         0.0    0.0   

       "\n\n#pfizer  "\n\n#realtimecovid19  "\n\n#vaccine  ...    🥺    🦠   🦠💉  \
0               0.0                    0.0            0.0  ...  0.0  0.0  0.0   
1               0.0                    0.0            0.0  ...  0.0  0.0  0.0   
2               0.0                    0.0            0.0  ...  0.0  0.0  0.0   
3               0.0                    0.0            0.0  ...  0.0  0.0  0.0   
4               0.0                    0.0            0.0  ...  0.0  0.0  0.0   
...             ...                    ...            ...  ...  ...  ...  ...   
80099           0.0                    0.0            0.0  ...  0.0  0.0  0.0   
80100           0.0                    0.0            0.0  ...  0.0  0.0  0.0   
80101           0.0                    0.0            0.0  ...  0.0  0.0  0.0   
80102           0.0                    0.0            0.0  ...  0.0  0.0  0.0   
80103           0.0                    0.0            0.0  ...  0.0  0.0  0.0   

         🧐  🧟\u200d♀️    🧠    🧡    🧬    🧵    🩹  
0      0.0        0.0  0.0  0.0  0.0  0.0  0.0  
1      0.0        0.0  0.0  0.0  0.0  0.0  0.0  
2      0.0        0.0  0.0  0.0  0.0  0.0  0.0  
3      0.0        0.0  0.0  0.0  0.0  0.0  0.0  
4      0.0        0.0  0.0  0.0  0.0  0.0  0.0  
...    ...        ...  ...  ...  ...  ...  ...  
80099  0.0        0.0  0.0  0.0  0.0  0.0  0.0  
80100  0.0        0.0  0.0  0.0  0.0  0.0  0.0  
80101  0.0        0.0  0.0  0.0  0.0  0.0  0.0  
80102  0.0        0.0  0.0  0.0  0.0  0.0  0.0  
80103  0.0        0.0  0.0  0.0  0.0  0.0  0.0  

[80104 rows x 14966 columns]

In [22]:
#Centres vectorized documents by subtracting the mean
tfidf_docs_df_mean_subtracted = tfidf_docs_df - tfidf_docs_df.mean()

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
svd = TruncatedSVD(n_components =20 , n_iter = 100) 
svd_topic_vectors = svd.fit_transform(tfidf_docs_df.values)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
svd
#svd_topic_vectors

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TruncatedSVD(n_components=20, n_iter=100)

In [25]:
#put it in a dataframe, again being good to ourselves by giving it row and column names
svd_topic_vectors_df = pd.DataFrame(svd_topic_vectors, index=df.index, columns=['topic{}'.format(i) for i in range(20)])
svd_topic_vectors_df

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


topic0    topic1    topic2    topic3    topic4    topic5    topic6  \
0      0.240449 -0.178909  0.065561  0.166963  0.136558  0.108987 -0.014748   
1      0.126301  0.110814 -0.116148 -0.025154  0.053078 -0.004095  0.026939   
2      0.088868  0.005339 -0.030490  0.047645 -0.097512 -0.027817 -0.138656   
3      0.081434  0.037799 -0.017071  0.001699 -0.003518 -0.013123  0.004330   
4      0.079521  0.029853 -0.010991  0.002257 -0.007435 -0.014920 -0.002521   
...         ...       ...       ...       ...       ...       ...       ...   
80099  0.062373  0.031038 -0.019391 -0.001096  0.000686 -0.020376 -0.002376   
80100  0.073301  0.045334 -0.023852  0.004002  0.002823 -0.026949  0.007734   
80101  0.103293  0.051534 -0.046233 -0.010308  0.014529 -0.002529  0.000329   
80102  0.066901  0.033327 -0.027598  0.006470 -0.014366 -0.031507 -0.023353   
80103  0.065045  0.041902 -0.054438 -0.006508 -0.002845  0.000345 -0.021534   

         topic7    topic8    topic9   topic10   topic11   topic12   topic13  \
0     -0.092903  0.017072 -0.044826 -0.005820 -0.031023  0.048458 -0.053059   
1     -0.036593  0.011940 -0.011096  0.048772 -0.053597 -0.017022  0.017088   
2      0.057085  0.032271 -0.031128 -0.025664  0.023159  0.024226 -0.061950   
3     -0.043553  0.008588 -0.011299  0.028679  0.039182  0.012525 -0.035055   
4     -0.044674  0.012539 -0.011653  0.028387  0.039662  0.012954 -0.015100   
...         ...       ...       ...       ...       ...       ...       ...   
80099 -0.043804  0.012622  0.014169 -0.019198 -0.019037  0.042665 -0.027164   
80100 -0.074783  0.017591 -0.012763  0.037188  0.012715  0.033964  0.031528   
80101 -0.021389  0.015510 -0.025517  0.058091  0.030588 -0.029445 -0.007492   
80102 -0.026164  0.021400  0.002876 -0.009298 -0.023925  0.032744 -0.019762   
80103  0.019374  0.013061 -0.024345  0.035411 -0.006468 -0.042909 -0.005387   

        topic14   topic15   topic16   topic17   topic18   topic19  
0     -0.082183  0.040217 -0.016416  0.062376  0.041411 -0.037446  
1      0.033972 -0.059363  0.001716  0.032396  0.061200 -0.054376  
2     -0.068221  0.011263  0.022315  0.067850  0.017766 -0.088263  
3     -0.017424  0.019006 -0.010316 -0.020883 -0.017458 -0.003565  
4     -0.006581  0.012711 -0.012371 -0.020923 -0.010622  0.007136  
...         ...       ...       ...       ...       ...       ...  
80099  0.003987 -0.007287  0.004217 -0.053757 -0.015910  0.075175  
80100 -0.022984 -0.020951 -0.033323  0.028793 -0.025282  0.019795  
80101  0.000591  0.005150 -0.012431 -0.009549  0.008177  0.005556  
80102 -0.014289 -0.019629  0.007198 -0.033852 -0.025874  0.072910  
80103 -0.003313 -0.026116  0.013676  0.003234 -0.015994 -0.017471  

[80104 rows x 20 columns]

In [26]:
#let's stick this in a friendly dataframe
#below, we use .T to transpose svd.components_ into the familiar form where *rows* correspond to words and *columns* to topics
topic_weights = pd.DataFrame(svd.components_.T, index=terms, columns=['topic{}'.format(i) for i in range(20)])
topic_weights #display it

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


topic0    topic1    topic2    topic3    topic4    topic5  \
           0.447950  0.128257  0.015260  0.007127 -0.025209 -0.078315   
"          0.006310  0.009865 -0.008086  0.000140  0.002377 -0.002349   
"#pfizer   0.000374  0.000717 -0.000729  0.000040  0.000020 -0.000309   
"#pfizer"  0.000121  0.000144 -0.000046 -0.000157  0.000105  0.000011   
"\n        0.000160  0.000190 -0.000138  0.000074  0.000184  0.000091   
...             ...       ...       ...       ...       ...       ...   
🧠          0.000111  0.000039 -0.000140  0.000132 -0.000011  0.000060   
🧡          0.000232  0.000213  0.000805  0.000041  0.000538 -0.000513   
🧬          0.000334  0.000066 -0.000106 -0.000048 -0.000042  0.000308   
🧵          0.000235  0.000333 -0.000404  0.000088 -0.000094 -0.000086   
🩹          0.000287  0.000005  0.000718 -0.000201 -0.000077 -0.000250   

             topic6    topic7    topic8    topic9   topic10   topic11  \
           0.038774 -0.319167  0.061936 -0.053312  0.237677  0.317673   
"         -0.000836 -0.002375  0.001379 -0.005136 -0.000220 -0.001991   
"#pfizer  -0.000286  0.000291  0.000157 -0.000060 -0.000431 -0.000421   
"#pfizer"  0.000129  0.000134  0.000076 -0.000229  0.000251 -0.000172   
"\n       -0.000120  0.000146 -0.000018  0.000107  0.000169  0.000365   
...             ...       ...       ...       ...       ...       ...   
🧠         -0.000153  0.000182 -0.000209  0.000024 -0.000222  0.000020   
🧡         -0.000142  0.000133  0.000045 -0.000387  0.000127  0.000415   
🧬          0.000027  0.000132  0.001152 -0.000146  0.000200 -0.000304   
🧵         -0.000445 -0.000081  0.000395 -0.000281  0.000040 -0.000304   
🩹          0.000347  0.000144 -0.000156 -0.000303 -0.000407 -0.000296   

            topic12   topic13   topic14   topic15   topic16   topic17  \
           0.128777 -0.109567 -0.028572  0.147661 -0.104043 -0.183649   
"         -0.005306 -0.005743 -0.006613 -0.000063 -0.000498  0.002697   
"#pfizer   0.000502 -0.000667 -0.000305 -0.000700  0.000223 -0.000593   
"#pfizer" -0.000413 -0.000256  0.000060  0.000013 -0.000179  0.000296   
"\n        0.000223 -0.000018 -0.000573  0.000134 -0.000019  0.000567   
...             ...       ...       ...       ...       ...       ...   
🧠          0.000154 -0.000018  0.000172 -0.000267 -0.000174  0.000263   
🧡          0.000121  0.000254  0.000492  0.000287  0.001299  0.000319   
🧬         -0.000437  0.000463  0.000452 -0.000355  0.000621  0.000028   
🧵         -0.000031  0.000373 -0.000256 -0.000517 -0.000264  0.000570   
🩹          0.000068  0.000644  0.000288  0.000345 -0.000805 -0.000135   

            topic18   topic19  
          -0.006058  0.106296  
"         -0.004022 -0.004475  
"#pfizer  -0.000530  0.000327  
"#pfizer" -0.000024 -0.000089  
"\n        0.000602 -0.000305  
...             ...       ...  
🧠         -0.000112  0.000078  
🧡         -0.000393  0.000407  
🧬          0.000018 -0.000528  
🧵         -0.000494 -0.000214  
🩹         -0.000438 -0.000655  

[14966 rows x 20 columns]

In [27]:
topic_weights.topic2.sort_values(ascending=False)[:20] # show top 10 weighted words for topic 2

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


vaccinated          0.586006
fully               0.501178
got                 0.197502
#vaccinated         0.111998
dose                0.087516
today               0.070625
💉                   0.070421
second              0.053526
i’m                 0.047621
just                0.040914
finally             0.039268
vaxxed              0.037117
officially          0.034603
#getvaccinated      0.023135
#fullyvaccinated    0.021424
i'm                 0.017455
2nd                 0.015388
                    0.015260
💉\n#pfizer          0.014817
\n#pfizer           0.014224
Name: topic2, dtype: float64

In [28]:
cv = CountVectorizer(min_df=5, tokenizer=my_tokeniser,)
count_array = cv.fit_transform(df['text'].values.astype('U'))
count_array.shape

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(80104, 14966)

In [29]:
lda = LatentDirichletAllocation(n_components=20,
                                random_state=123,
                                learning_method='batch')
lda.fit(count_array)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=20, random_state=123)

In [30]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(count_array)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=20, random_state=0)

In [31]:
# And run LDA. This could take a loooonggggg time....
lda_topics = lda.fit_transform(count_array)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### <p style="font-family:Garamond,serif"> Generate visualitaion with pyLDAvis </p>

In [32]:
pyLDAvis.sklearn.prepare(lda, count_array, cv)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.220728 -0.080242       1        1  10.757925
17    -0.202648 -0.019168       2        1   9.945037
1      0.119608 -0.113105       3        1   7.546765
19     0.011440  0.130335       4        1   6.831567
4      0.146716 -0.047238       5        1   6.248591
18     0.140243  0.079936       6        1   5.790410
14    -0.167329 -0.005368       7        1   5.241613
6      0.038841 -0.174210       8        1   5.037732
15     0.150378 -0.067388       9        1   4.679243
11     0.055305 -0.160275      10        1   3.862567
13     0.056589  0.094594      11        1   3.759102
3      0.033979  0.009855      12        1   3.556003
10     0.113820  0.114424      13        1   3.554380
16    -0.157434  0.048344      14        1   3.468269
9     -0.000065 -0.043411      15        1   3.411091
7      0.027171  0.074561      16        1   3.353332
5     -0.052845  0.053576      17        1   3.308705
2      0.006006  0.155826      18        1   3.287090
12     0.000795 -0.052698      19        1   3.199775
8     -0.099843  0.001650      20        1   3.160805, topic_info=                Term          Freq         Total Category  logprob  loglift
13864     vaccinated   6493.000000   6493.000000  Default  30.0000  30.0000
6986           covid   8846.000000   8846.000000  Default  29.0000  29.0000
8603             got  11128.000000  11128.000000  Default  28.0000  28.0000
3218              19   3921.000000   3921.000000  Default  27.0000  27.0000
8442           fully   3226.000000   3226.000000  Default  26.0000  26.0000
...              ...           ...           ...      ...      ...      ...
665    #covidvaccine    162.959049   5787.452420  Topic20  -5.4949  -0.1156
9637            just    167.469010   8233.613494  Topic20  -5.4676  -0.4408
14065             wa    158.766546   6958.921730  Topic20  -5.5210  -0.3260
9528             jab    146.510269   4782.386749  Topic20  -5.6013  -0.0313
10549           need    133.524144   2868.235214  Topic20  -5.6941   0.3872

[1622 rows x 6 columns], token_table=       Topic      Freq Term
term                       
0          1  0.182891     
0          2  0.082751     
0          3  0.075508     
0          4  0.055326     
0          5  0.060143     
...      ...       ...  ...
14868     10  0.076256    🙌
14868     14  0.655801    🙌
14868     16  0.030502    🙌
14868     17  0.091507    🙌
14872      7  0.948483   🙌🏾

[5799 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 18, 2, 20, 5, 19, 15, 7, 16, 12, 14, 4, 11, 17, 10, 8, 6, 3, 13, 9])

In [ ]:
vis_en_hashtag_pfizer = pyLDAvis.sklearn.prepare(lda, count_array, cv)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### <p style="font-family:Garamond,serif"> Save the file into html for better viewing, please check the html file for both visualization. :)</p>

In [ ]:
pyLDAvis.save_html(vis_en_hashtag_pfizer, "vis_en_hashtag_pfizer.html")